# Continuous Optimisation HW2

In [ ]:
# Imports
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from tqdm import tqdm
from scipy.io import loadmat


data = loadmat('data-toy.mat')
print(data.keys())
print(data['K'])
sigma = float(data['sigma'])

'''
K = data['K']
P = data['P']
X0 = data['X0']
d = data['d']
delta_0 = data['delta_0']
delta_bar = data['delta_bar']
n = data['n']
sigma = data['sigma']
y = data['y']
'''

sigmasquared = sigma**2


# Question 1
Implement phi(x, P), bigphi(X, P), f(X, P, y).

In [ ]:
# Question 1
#TODO Vectorise all the functions

def h(x: np.ndarray) -> float:
    """
    Gaussian filter

    :x: np.ndarray[(1, 2)]
    :returns: float
    """
    return np.e**(-np.inner(x, x)/sigmasquared)  # Always take sigma = 0.1


def phi(x,P):
    """
    Calculate contribution of each 'true' star to observed image
    
    :x: np.ndarray[(2, 1)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(n**2, 1)]
    """
    # print(f"P is {P}")
    # print(f"x is {x}")
    # print(f"P-x is {P-x}")
    # print(h(P - x).reshape(-1, 1))
    # print((np.e**((-1/sigmasquared)*np.sum((P.T-x.T)**2,axis=1))).reshape(-1, 1))
    # return h(P - x).reshape(-1, 1)
    return (np.e**((-1/sigmasquared)*np.sum((P.T-x.T)**2,axis=1))).reshape(-1, 1)


def bigphi(X, P):
    """
    Calculate image observed, based on K-star positions X
    
    :X: np.ndarray[(2, K)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(n**2, 1)]
    """
    global K, n
    #non-vectorised code just in case we need it
    bigphi = np.zeros((n**2, 1))
    for i in range(K):
         bigphi += phi(X[2*i:2*i+2], P)
    #return phi(X, P)
    return bigphi


def  f(X, P, y):
    """
    Calculate squared error of estimate bigphi(X)
    
    :X: np.ndarray[(2, K)]
    :P: np.ndarray[(2, n**2)]
    :y: np.ndarray[(n**2, 1)]
    :returns: float
    """
    global K, n
    return (1/(2*n**2)) * np.linalg.norm(bigphi(X, P)-y)**2


# Question 2

We see that $f$ is not convex. There are clear local maxima which cannot occur if $f$ were convex.

In [ ]:
K = 1
n = 2
true_positions = np.array([[0], [0]])
positions = np.array([[0.5, 0.5], [-0.5, 0.5], [-0.5, -0.5], [0.5, -0.5]]).T
y = np.array([[0], [0], [1], [0]])
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

x_vals = np.linspace(-1, 1, 100)
y_vals = np.linspace(-1, 1, 100)
x_vals, y_vals = np.meshgrid(x_vals, y_vals)
grid = np.array([x_vals, y_vals]).reshape((2, 100**2))
z_vals = np.zeros((1, 10000))
for i in range(10000):
    z_vals[0, i] = f(grid[:, i].reshape((2, 1)), positions, y)
z_vals = z_vals.reshape((100, 100))
ax.scatter(x_vals, y_vals, z_vals)
plt.show()
plt.show()

# Question 6.1

In [ ]:
def d_phi_i_0(x,p_i):
    """
    Computes value of cell i_0 of jacobian of phi
    
    :x: np.ndarray[(2, 1)]
    :p_i: np.ndarray[(2, 1)]
    :returns: float
    """
    return (2/sigmasquared)*(p_i[0]-x[0])*np.e**((-1/sigmasquared)*np.inner(p_i-x,p_i-x))

def d_phi_i_1(x,p_i):
    """
    Computes value of cell i_1 of jacobian of phi
    
    :x: np.ndarray[(2, 1)]
    :p_i: np.ndarray[(2, 1)]
    :returns: float
    """
    return (2/sigmasquared)*(p_i[1]-x[1])*np.e**((-1/sigmasquared)*np.inner(p_i-x,p_i-x))
    
def d_phi(x, P):
    """
    Computes the jacobian of small phi

    :x: np.ndarray[(1, 2)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(n**2, 2)]
    """
    # Subtract x from each column of P
    diff = P.T - x
    # Compute the squared differences
    squared_diff = np.sum(diff**2, axis=1)
    # Compute the exponential term for all elements at once
    exp_term = np.exp((-1 / sigmasquared) * squared_diff)
    # Compute the values for d_phi_i_0 and d_phi_i_1 using vectorized operations
    d_phi_0 = (2 / sigmasquared) * diff[:, 0] * exp_term
    d_phi_1 = (2 / sigmasquared) * diff[:, 1] * exp_term
    # Stack the results to form the final array
    d_phi = np.stack((d_phi_0, d_phi_1), axis=1)
    return d_phi

def d_big_phi(X,P):
    """
    Computes the jacobian of big phi

    :X: np.ndarray[(1, 2K)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(n**2, 2K)]
    """
    d_big_phi = np.zeros((n**2,2*K))
    for i in range(K):
        d_big_phi[0:n**2,2*i:2*i+2] = d_phi(X[2*i:2*i+2].T,P)
    return d_big_phi

def d_f(X,P):
    """
    Computes the jacobian of f

    :X: np.ndarray[(1, 2K)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(1, 2K)]
    """
    return (1/n**2)*(bigphi(X,P)-y).T@d_big_phi(X,P)

#compute gradient of f
def grad_f(X,P):
    """
    Computes the gradient of f

    :X: np.ndarray[(1, 2K)]
    :P: np.ndarray[(2, n**2)]
    :returns: np.ndarray[(2K, 1)]
    """
    return d_f(X,P).T

# Question 6.2

In [ ]:
#checking gradient is correct numerically
K = int(data['K'])
P = data['P']
X0 = data['X0']
d = int(data['d'])
delta_0 = data['delta_0']
delta_bar = data['delta_bar']
n = int(data['n'])
sigma = float(data['sigma'])
y = data['y'].flatten(order='F').reshape(n**2,1)

#check that the gradient is correct using
# f(x+tv)= f(x) + t<v,grad_f(x)> + O(t^2)

# Generate a random point and a random direction
theta = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)
v = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)
v = v / np.linalg.norm(v)

## Check the gradient 
def checkgradient(f,grad_f, theta,v):
    #logspace of t values
    t=np.logspace(-8, 0, num=100)
    #intialise error to 0
    error = np.zeros_like(t)
    #pre-calculae f_lambda and f_lambda_grad to use in for loop
    f_lambda = f(theta,P,y)
    f_lambda_grad = grad_f(theta,P)
    #compute the error at each t
    for i in tqdm(range(100)):
        error[i] = np.abs( f(theta+(t[i]*v),P,y)-f_lambda-(t[i]*v.T@f_lambda_grad) )
        
    #plot the graph of error vs t
    plt.loglog(t,error)
    plt.xlabel('t (log scale)')
    plt.ylabel('Error (log scale)')
    plt.title('Plot of t vs Error')
    plt.grid()
    plt.show
checkgradient(f,grad_f,theta,v)

# Question 7

## Computing the Hessian 

First we re-derive the gradient of f:

$$ f(x) = \frac{1}{2n^2}||\Phi(x) - y ||^2$$

$$ Df(x)[v] = \frac{1}{n^2}< \Phi(x) - y,D \Phi(x)[v] > = \frac{1}{n^2}<(D\Phi(x))^*[\Phi(x) - y],v> $$

$$ \nabla f(x) = \frac{(D\Phi(x))^*[\Phi(x) - y]}{n^2} $$

Then, using the above calculations, we can calculate the hessian:

$$ \frac{1}{n^2}[D((D\Phi(x))^*[\Phi(x) - y])[v]] = \frac{1}{n^2}[D((D\Phi(x))^*[v])[\Phi(x) - y] + (D\Phi(x))^*[D\Phi(x)[v]]]$$


$$ \nabla^2 f(x)[v] = \frac{1}{n^2}[(D(D\Phi(x))^*[v])[\Phi(x)-y]+D\Phi(x)^*[D\Phi(x)[v]]] $$

The implicit reason why we are calculating the hessian with respect to a direction v is because it is computationally much cheaper than to calculate the full hessian which means calculating all 2nd order partial derivatives of f.



In [ ]:
def d_g(x,p,v):
    """
    Computes directional derivative of g_1, g_2 at x in the direction v

    :param x: np.ndarray[(2, 1)] "position of a stat"
    :param p: np.ndarray[(2, 1)] "pixel positions"
    :param v: np.ndarray[(2, 1)] "vector direction of change in star position"
    :returns: scalar
    """
    d_g_1 = np.zeros((1,2))
    d_g_1[0][0] = ((4/sigmasquared**2)*((p[0]-x[0])**2)*np.e**(-(p-x).T@(p-x)/sigmasquared))-(2/sigmasquared)*np.e**(-(p-x).T@(p-x)/sigmasquared)
    d_g_1[0][1] = (4/sigmasquared**2)*(p[0]-x[0])*(p[1]-x[1])*np.e**(-(p-x).T@(p-x)/sigmasquared)

    d_g_2 = np.zeros((1,2))
    d_g_2[0][0] = d_g_1[0][1]
    d_g_2[0][1] = ((4/sigmasquared**2)*((p[1]-x[1])**2)*np.e**(-(p-x).T@(p-x)/sigmasquared))-(2/sigmasquared)*np.e**(-(p-x).T@(p-x)/sigmasquared)

    return d_g_1@v,d_g_2@v


def d_d_phi(X, P, V):
    """
    Compute the directional derivative of the Jacobian of phi at x in the direction v.

    :param X: np.ndarray[(1, 2K)] "position of a stat"
    :param P: np.ndarray[(2, n**2)] "pixel positions"
    :param V: np.ndarray[(1, 2K)] "vector direction of change in star position"
    :returns: np.ndarray[(n**2, 2K)]
    """
    d_d_phi = np.zeros((n**2,2*K))
    for i in range(K):
        for j in range(n**2):
            #puts p_j in an array
            p_j = np.array(
                [
                 [P[0][j]],
                 [P[1][j]]
                 ]
                );
            d_d_phi[j,2*i],d_d_phi[j,2*i+1] = d_g(X[2*i:2*i+2],p_j,V[2*i:2*i+2])
    return d_d_phi
    


def hessian_f(X,V):
    """
    Compute the Hessian of f at X in the direction v.
    
    :param X: np.ndarray[(2, K)] "position of K stars"
    :param P: np.ndarray[(2, n**2)] "pixel positions"
    :param y: np.ndarray[(n**2, 1)] "actual image detected, y = \Phi(X_true)" remember we want to find X_true
    :param v: np.ndarray[(2, K)] "direction at which hessian is taken" 
    :returns: np.ndarray[(2*K, 1)]
    """

    # Compute intermediate terms
    phi_X = bigphi(X, P) - y  # (n**2, 1)
    d_big_phi_X = d_big_phi(X, P)  # (n**2, 2*K)
    d_big_phi_X_v = d_big_phi(X, P) @ V  # directional derivative of big_phi in direction v

    # Compute each term in the Hessian formula

    term1 = d_d_phi(X, P, V).T @ phi_X 
    term2 = d_big_phi_X.T @ d_big_phi_X_v 

    hessian = (1 / n**2) * (term1 + term2)
    return hessian



# Question 8


In [ ]:

X_random = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)  # Random X
U_random = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)  # Random direction U
U_random /= np.linalg.norm(U_random)  # Normalize U

#check that the gradient is correct using
# f(x+tv)= f(x) + t<v,grad_f(x)> + t^2/2*v.T@hess_f(x)[v] + O(t^3)

# Generate a random point and a random direction
theta = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)
v = np.random.uniform(-0.5, 0.5, (d, K)).flatten(order='F').reshape(d*K,1)
v = v / np.linalg.norm(v)

## Check the hessian 
def checkhessian(f,grad_f,hessian_f,theta,v):
    #logspace of t values
    t=np.logspace(-8, 0, num=100)
    #intialise error to 0
    error = np.zeros_like(t)
    #pre-calculae f_lambda and f_lambda_grad to use in for loop
    f_lambda = f(theta,P,y)
    f_lambda_grad = grad_f(theta,P)
    f_lambda_hess_v = hessian_f(theta,v)
    print(f_lambda_hess_v.shape)
    #compute the error at each t
    for i in tqdm(range(100)):
        error[i] = np.abs(f(theta+(t[i]*v),P,y)-f_lambda-(t[i]*v.T@f_lambda_grad)- ((t[i]**2)/2)*v.T@f_lambda_hess_v)
        
    #plot the graph of error vs t
    plt.loglog(t,error)
    plt.xlabel('t (log scale)')
    plt.ylabel('Error (log scale)')
    plt.title('Plot of t vs Error')
    plt.grid()
    plt.show
checkhessian(f,grad_f,hessian_f,theta,v)

# Question 9

In [ ]:
def tCG(H: np.ndarray, b: np.array, radius: float) -> tuple:
    v0, r0, p0 = 0, b, b  # Here, when there is a "0" after the variable, it means it's the "iterated" variable ie it changes on every loop
    while True:
        Hp = H @ p0
        inner = np.inner(p0, Hp)
        alpha = np.inner(r0, r0)/inner
        v_plus = v0 + alpha * p0
        if inner <= 0 or np.linalg.norm(v_plus) >= radius:
            inner_pv = np.inner(v0, p0)
            norm2_p = np.inner(p0, p0)
            t = (-inner_pv + np.sqrt(inner_pv - (np.inner(v0, v0) - radius**2) * norm2_p))/norm2_p
            v0 += t * p0
            return (v0, b - r0 + t*Hp, True)  # since we solved ||v0|| = Delta, the flag is true
        else:
            v0 = v_plus
        r_old = r0
        r0 -= alpha * Hp
        if np.norm(r0) <= np.norm(b) * min(np.norm(r0), 0.1):
            return (v0, b - r0, False)  # flag is false
        beta0 = np.inner(r0, r0)/np.inner(r_old, r_old)
        p0 = r0 + beta0 * p0


# Question 10

In [ ]:
def TRsolver(x0: np.array, Delta0: float, DeltaBar: float, rhoPrime: float=0.1) -> np.array:
    TRsub = tCG(hessian_f())
